In [78]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import uuid
import shutil
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img , img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [79]:
tf.keras.backend.clear_session()

## Data Augmentation

In [80]:
# # Define data augmentation parameters (adjust as needed)
# datagen = ImageDataGenerator(
#     rotation_range=25,  # Rotate images by up to 40 degrees
#     shear_range=0.25,     # Shear by up to 20%
#     zoom_range=0.25,      # Zoom by up to 20%
#     horizontal_flip=True, # Randomly flip images horizontally
#     brightness_range=(0.5, 1.5)  # Adjust brightness range
# )

# # Define path to your image folder
# image_dir = "Train/SwivelChair"

# # Define output directory for augmented images (create if it doesn't exist)
# save_to_dir = "Train/SwivelChair"
# import os
# os.makedirs(save_to_dir, exist_ok=True)  # Create directory if it doesn't exist

# # Loop through each image in the folder
# for filename in os.listdir(image_dir):
#     # Load the image
#     img = load_img(os.path.join(image_dir, filename))

#     # Convert to array
#     x = img_to_array(img)

#     # Reshape (optional, based on your model's requirements)
#     x = x.reshape((1,) + x.shape)  # Reshape if your model expects a specific format

#     # Generate and save augmented images
#     i = 0
#     for batch in datagen.flow(x, batch_size=1, save_to_dir=save_to_dir, save_prefix="aug_", save_format="jpeg"):
#         i += 1
#         # You can adjust the number of augmented images per original image here (e.g., i > 5)
#         if i > 2:
#             break
# print(f"Successfully augmented and saved images to {save_to_dir}")


## Modeling

In [81]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

In [82]:
x = tf.keras.layers.GlobalAvgPool2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(5,activation="softmax")(x)

model = tf.keras.Model(inputs = base_model.input , outputs = x)


In [83]:
#freezing the original model's layers
for layer in base_model.layers:
    layer.trainable = False

In [84]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=CategoricalCrossentropy(from_logits=True),  # Adjust if needed
              metrics=['accuracy'])

In [85]:
train_Path = "Train"
IMAGE_SIZE = (224,224)
folders = glob.glob(train_Path + '/*')
folders

['Train\\Bed',
 'Train\\Chair',
 'Train\\Sofa',
 'Train\\SwivelChair',
 'Train\\Table']

In [217]:
labels = ["Bed","Chair","Sofa","SwivelChair","Table"]

In [87]:
train_datagen = ImageDataGenerator(
    rescale = 1./225,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
training_set = train_datagen.flow_from_directory(
    train_Path,
    target_size = IMAGE_SIZE,
    batch_size = 32,
    class_mode = 'categorical', # As we have more than 2 so using categorical.. for 2 we might have used binary.
    shuffle = True
)


Found 9829 images belonging to 5 classes.


In [88]:
test_datagen = ImageDataGenerator(rescale=1./225)
test_Path = "Test"
test_set = test_datagen.flow_from_directory(
    test_Path,  # Use your test data directory here
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

Found 423 images belonging to 5 classes.


In [89]:
model.fit(training_set, epochs=10, steps_per_epoch=len(training_set),validation_data=test_set)

Epoch 1/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 210s 654ms/step - accuracy: 0.8016 - loss: 0.5158 - val_accuracy: 0.9267 - val_loss: 0.2053
Epoch 2/10


c:\Users\ghass\anaconda3\envs\yolo\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


308/308 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9267 - val_loss: 0.2069
Epoch 3/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 204s 655ms/step - accuracy: 0.9413 - loss: 0.1761 - val_accuracy: 0.9433 - val_loss: 0.1366
Epoch 4/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9433 - val_loss: 0.1357
Epoch 5/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 201s 642ms/step - accuracy: 0.9511 - loss: 0.1332 - val_accuracy: 0.9574 - val_loss: 0.1318
Epoch 6/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9574 - val_loss: 0.1235
Epoch 7/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 196s 628ms/step - accuracy: 0.9609 - loss: 0.1154 - val_accuracy: 0.9574 - val_loss: 0.1617
Epoch 8/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9574 - val_loss: 0.1678
Epoch 9/10
308/308 ━━━━━━━━━━━━━━━━━━━━ 197s 632ms/step - accuracy:

In [208]:
pred_test = model.predict(test_set, steps=len(test_set))

14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 366ms/step


In [209]:
predictions = np.argmax(pred_test,axis=1)

In [210]:
len(predictions)

423

In [211]:
predictions

array([4, 0, 3, 2, 2, 0, 1, 0, 0, 1, 2, 1, 3, 3, 0, 1, 0, 1, 1, 0, 3, 1,
       1, 1, 1, 2, 1, 1, 0, 3, 1, 1, 3, 1, 2, 2, 2, 3, 2, 3, 2, 0, 1, 1,
       2, 3, 1, 3, 0, 0, 1, 1, 2, 0, 3, 1, 0, 0, 2, 1, 0, 2, 3, 1, 1, 2,
       0, 3, 3, 1, 3, 0, 1, 0, 3, 1, 2, 2, 2, 1, 2, 2, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 2, 0, 1, 0, 1, 0, 4, 1, 1, 0, 0, 2, 0, 3, 0, 1, 3, 0,
       3, 0, 2, 3, 3, 2, 4, 0, 2, 3, 3, 2, 1, 3, 3, 0, 4, 0, 3, 0, 0, 0,
       0, 2, 4, 1, 2, 3, 1, 2, 0, 0, 0, 1, 0, 2, 0, 3, 0, 1, 2, 3, 3, 1,
       1, 2, 3, 0, 3, 2, 1, 0, 4, 3, 1, 1, 0, 0, 4, 0, 3, 3, 1, 2, 2, 2,
       2, 4, 3, 1, 2, 1, 3, 3, 3, 1, 0, 2, 2, 1, 1, 1, 2, 3, 2, 1, 0, 2,
       2, 2, 0, 3, 3, 0, 0, 1, 3, 2, 2, 1, 1, 1, 1, 2, 2, 1, 0, 2, 2, 3,
       0, 3, 2, 0, 2, 3, 3, 1, 3, 1, 0, 1, 4, 0, 2, 4, 0, 3, 4, 1, 4, 1,
       1, 0, 1, 3, 1, 0, 2, 0, 0, 2, 3, 0, 3, 2, 2, 0, 2, 3, 1, 2, 4, 1,
       1, 4, 2, 3, 1, 0, 2, 3, 2, 3, 3, 3, 3, 3, 0, 1, 3, 0, 3, 0, 2, 2,
       3, 1, 3, 2, 1, 1, 1, 4, 0, 1, 1, 2, 0, 2, 2,

In [212]:
labels = np.empty(0)  # Initialize an empty array
for i in range(14):
    label = np.argmax(test_set[i][1], axis=1)  # Extract predictions
    labels = np.concatenate((labels, label))

In [213]:
labels

array([0., 1., 3., 4., 2., 0., 2., 3., 0., 3., 2., 3., 1., 3., 3., 0., 0.,
       1., 1., 3., 1., 3., 2., 3., 1., 3., 2., 2., 2., 2., 0., 4., 1., 0.,
       3., 0., 2., 2., 3., 0., 2., 2., 1., 0., 3., 1., 1., 0., 0., 1., 4.,
       4., 3., 1., 1., 0., 1., 3., 4., 2., 0., 2., 1., 2., 3., 2., 1., 1.,
       3., 0., 4., 0., 0., 3., 2., 3., 0., 0., 1., 3., 0., 2., 1., 1., 1.,
       2., 2., 2., 4., 3., 3., 0., 3., 2., 0., 1., 0., 2., 0., 0., 3., 1.,
       2., 2., 0., 0., 1., 1., 1., 3., 0., 3., 0., 1., 2., 3., 2., 3., 3.,
       3., 2., 3., 3., 2., 0., 3., 3., 0., 0., 1., 3., 1., 4., 0., 2., 1.,
       3., 4., 0., 0., 0., 0., 1., 0., 0., 2., 3., 1., 0., 2., 0., 3., 3.,
       1., 2., 0., 1., 0., 0., 0., 3., 2., 2., 0., 1., 3., 3., 1., 3., 4.,
       3., 1., 0., 1., 3., 1., 4., 2., 3., 2., 3., 2., 2., 4., 3., 0., 3.,
       3., 3., 3., 2., 1., 2., 2., 0., 2., 2., 1., 4., 1., 1., 3., 2., 2.,
       3., 0., 2., 0., 2., 1., 0., 4., 3., 2., 2., 0., 0., 3., 2., 0., 2.,
       0., 0., 2., 3., 2.

In [214]:
#Overall 
accuracy_metric = tf.keras.metrics.Accuracy()
accuracy_metric.update_state(labels,predictions)
accuracy = accuracy_metric.result().numpy()
print("Accuracy:", accuracy)

Accuracy: 0.23640662


In [320]:
# Define path to your image
image_path = "swivelchair (20).jpg"

# Load the image
img = load_img(image_path, target_size=IMAGE_SIZE)  # Adjust if needed

# Convert to array and normalize
x = img_to_array(img)
x = np.expand_dims(x, axis=0)  # Add a batch dimension as the model expects batches
x = x / 255.0  # Normalize pixel values (assuming rescale used during training)


In [321]:
# Make predictions
predictions = model.predict(x)

# Get the class with the highest probability
predicted_class = np.argmax(predictions[0])  # Assuming one prediction per image
max = np.max(predictions[0])
if(max<0.7):
    print("Other")
else:
    print(labels[predicted_class])
print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
SwivelChair
[[2.9903550e-08 1.5067562e-05 2.1679769e-07 9.9998474e-01 4.9781217e-08]]


In [270]:
# model.save("FurnitureModel.keras")

In [ ]:
sub_models ={}
def sub_classify(main_class,img):
    
    if(main_class == "Furniture"):
        sub_models["Furniture"] = tf.keras.models.load_model('BestModels/FurnitureModel.keras')
        predictions = sub_models["Furniture"].predict(img)
        class_names = ["Bed","Chair","Sofa","SwivelChair","Table"]
        max = np.argmax(predictions[0])
        if(max<0.8):
            predicted_class = "Other"
        else:
            predicted_class = class_names[max]
        return {"main_class" : "Furniture" , "sub_class": predicted_class}
    
    elif main_class == 'It':
        sub_models["It"] = tf.keras.models.load_model('BestModels/ItModelV1.keras')
        predictions = sub_models["It"].predict(img)
        class_names = ["Laptop","Printer","Samrtphone","Tv"]
        max = np.argmax(predictions[0])
        if(max<0.8):
            predicted_class = "Other"
        else:
            predicted_class = class_names[max]
        return {"main_class" : "It" , "sub_class": predicted_class}
    
    elif main_class == 'Jewellery':
        sub_models[main_class] = tf.keras.models.load_model('BestModels/JewelleryModelV1.keras')
        sub_models["Material"] = tf.keras.models.load_model('BestModels/MaterialModelV2.keras')
        jew_predictions = sub_models["Jewellery"].predict(img)
        mat_predictions = sub_models["Material"].predict(img)
        jew_class_names = ["earring","necklace","ring"]
        mat_class_names = ["gold","silver","bronze"]
        jew_max = np.argmax(jew_predictions[0])
        mat_max = np.argmax(mat_predictions[0])
        if(max<0.75):
            predicted_class = "Other"
        else:
            predicted_class = class_names[max]
        return {"main_class" : "Jewellery" , "sub_class": jew_class_names[jew_max] , "material": mat_class_names[mat_max] }
    
    elif main_class == 'Animal':
        sub_models[main_class] = tf.keras.models.load_model('BestModels/AnimalModel.keras')
        predictions = model.predict(img)
        decoded_prediction = tf.keras.applications.efficientnet_v2.decode_predictions(predictions, top=5)[0]
        return {"main_class" : "Animal" , "sub_class": decoded_prediction[0][1]}
    else:
        return {"main_class": main_class}